<a href="https://colab.research.google.com/github/Anirudddha08/Python-Programming/blob/master/Covid-Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
pip install streamlit_folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken 1t9O59m350kzw0iHZum6DhCJWVv_3haTpNavATH2Z7oG1KfPK

/bin/bash: line 1: ngrok: command not found


In [ ]:
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=85f00d3158cf7da24da34b28eb06979720cff903e160ef9a5b77e2a6cf5140fe
  Stored in directory: /root/.cache/pip/wheels/f4/e2/28/275496a22cb15fb495458aed2b164b4ccaa10e2f44188fb0b9
Successfully built pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_folium import folium_static
import json
import folium
import numpy as np
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize
import plotly
import plotly.express as px
import streamlit as st
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode,iplot, plot
import os
import pyarrow

st.markdown("<h1 style='text-align: center; color: #ff634d;'><strong><u>Covid-19 Dashboard</u></strong></h1>", unsafe_allow_html=True)
st.sidebar.markdown("<h1 style='text-align: center; color: #aaccee;'><strong><u>Covid-19 Dashboard</u></strong></h1>", unsafe_allow_html=True)

st.markdown("This Web App is a live Covid-19 Dashboard which access Data sourced from Johns Hopkins CSSE", unsafe_allow_html=True)




def main():

  conn = http.client.HTTPSConnection("api.covid19api.com")
  payload =''
  headers={}
  conn.request("GET","/summary",payload,headers)
  res = conn.getresponse()
  data=res.read().decode('UTF-8')
  covid = json.loads(data)


  df=pd.DataFrame(covid['Countries'])

  covid1=df.drop(columns=['CountryCode','Slug','Date','Premium'],axis=1)
  covid1['ActiveCases'] = covid1['TotalConfirmed']-covid1['TotalRecovered']
  covid1['ActiveCases'] = covid1['ActiveCases']-covid1['TotalDeaths']


  st.markdown("COVID-19 NUMBERS ACROSS THE GLOBE", unsafe_allow_html=True)
  dfn=covid1.drop(['NewConfirmed','NewDeaths','NewRecovered'],axis=1)
  dfn = dfn.groupby('Country')['TotalConfirmed','TotalDeaths','TotalRecovered','ActiveCases'].sum().sort_values(by='TotalConfirmed',ascending=False)
  dfn.style.background_gradient(cmap='Oranges')

  dfc = covid1.groupby('Country')['TotalConfirmed','TotalDeaths','TotalRecovered','ActiveCases'].max().sort_values(by='TotalConfirmed',ascending=False).reset_index()

  m = folium.Map(tiles='Stamen Terrain',min_zoom=1.5)
  url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
  country_shapes = f'{url}/world-countries.json'
  folium.Choropleth(
  geo_data=country_shapes,
  min_zoom=2,
  name='Covid-19',
  data=covid1,
  columns=['Country','TotalConfirmed'],
  key_on='feature.properties.name',
  fill_color='YlOrRd',
  nan_fill_color='black',
  legend_name='Total Confirmed Cases',).add_to(m)

  m1 = folium.Map(tiles='Stamen Terrain',min_zoom=1.5)
  url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
  country_shapes = f'{url}/world-countries.json'
  folium.Choropleth(
  geo_data=country_shapes,
  min_zoom=2,
  name='Covid-19',
  data=covid1,
  columns=['Country','TotalRecovered'],
  key_on='feature.properties.name',
  fill_color='YlOrRd',
  nan_fill_color='black',
  legend_name='Total Recovered Cases',).add_to(m1)

  m4 = folium.Map(tiles='Stamen Terrain',min_zoom=1.5)
  url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
  country_shapes = f'{url}/world-countries.json'
  folium.Choropleth(
  geo_data=country_shapes,
  min_zoom=2,
  name='Covid-19',
  data=covid1,
  columns=['Country','ActiveCases'],
  key_on='feature.properties.name',
  fill_color='YlOrRd',
  nan_fill_color='black',
  legend_name='Active Cases',).add_to(m4)


  coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
  covid_final = pd.merge(covid1,coordinates,on='Country')

  dfn
  confirmed_tot = int(dfc['TotalConfirmed'].sum())
  deaths_tot = int(dfc['TotalDeaths'].sum())
  recovered_tot = int(dfc['TotalRecovered'].sum())
  active_tot = int(dfc['ActiveCases'].sum())


  st.markdown("WORLD COVID-19 STATS", unsafe_allow_html=True)

  st.write('TOTAL CONFIRMED CASES FROM ALL OVER THE WORLD - ',confirmed_tot)
  st.write('TOTAL DEATH CASES FROM ALL OVER THE WORLD - ',deaths_tot)
  st.write('TOTAL RECOVERED CASES FROM ALL OVER THE WORLD - ',recovered_tot)
  st.write('TOTAL ACTIVE CASES FROM ALL OVER THE WORLD - ',active_tot)




  st.sidebar.subheader('Analysis through Map - Folium')

  select = st.sidebar.selectbox('Choose Map Type',['Confirmed Cases','Recovered Cases','Active Cases','Deaths'],key='1')


  if not st.sidebar.checkbox("Hide Map",True):

      if select == "Confirmed Cases":
        folium_static(m)
      elif select == "Recovered Cases":
        folium_static(m1)
      elif select == "Active Cases":
        folium_static(m4)
      else:
          m2 = folium.Map(tiles='StamenToner',min_zoom=1.5)
          deaths=covid_final['TotalDeaths'].astype(float)
          lat=covid_final['latitude'].astype(float)
          long=covid_final['longitude'].astype(float)

          m2.add_child(HeatMap(zip(lat,long,deaths),radius=0))
          folium_static(m2)

  st.sidebar.subheader('Analysis through Bar Chart - Plotly')

  select = st.sidebar.selectbox('Choose Bar Chart',['Confirmed Cases','Recovered Cases','Active Cases','Deaths'],key='2')



  if not st.sidebar.checkbox("Hide Bar Chart",True):
      if select == "Confirmed Cases":
          fig = px.bar(dfc.head(10), y='TotalConfirmed',x='Country',color='Country',height=400)
          fig.update_layout(title='Comparison of Total Confirmed Cases of 10 Most Affected Countries',xaxis_title='Country',yaxis_title='Total Confirmed Case',template="plotly_dark")
          st.plotly_chart(fig)
      elif select == "Recovered Cases":
          fig = px.bar(dfc.head(10), y='TotalRecovered',x='Country',color='Country',height=400)
          fig.update_layout(title='Comparison of Total Recovered of 10 Most Affected Countries',xaxis_title='Country',yaxis_title='Total Recovered',template="plotly_dark")
          st.plotly_chart(fig)
      elif select == "Active Cases":
          fig = px.bar(dfc.head(10), y='ActiveCases',x='Country',color='Country',height=400)
          fig.update_layout(title='Comparison of Active Cases of 10 Most Affected Countries',xaxis_title='Country',yaxis_title='Total Recovered',template="plotly_dark")
          st.plotly_chart(fig)
      else:
          fig = px.bar(dfc.head(10), y='TotalDeaths',x='Country',color='Country',height=400)
          fig.update_layout(title='Comparison of Total Deaths of 10 Most Affected Countries',xaxis_title='Country',yaxis_title='Total Deaths',template="plotly_dark")
          st.plotly_chart(fig)


  st.sidebar.subheader('Analysis through 3D Plot')

  if not st.sidebar.checkbox("Hide 3D plot",True):

      fig = px.scatter_3d(dfc.head(10), x='TotalConfirmed', y='TotalDeaths', z='TotalRecovered',
              color='Country')
      fig.update_layout(title='3D Plot of Total Cases, Total Deaths and Total Recovered of Top 10 Affected Countries')
      st.plotly_chart(fig)

  if st.sidebar.checkbox("Show Raw Data",False):
      st.subheader("Covid 19 Data")
      st.write(covid1)

def load_data():
    covid19 = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', encoding='ISO-8859-1',thousands='.', decimal=',', engine='python')
    covid19['date'] = pd.to_datetime(covid19['date'],format = '%Y-%m-%d')
    return covid19
covid19 = load_data()


###################################################################################

covid19['new_tests'] = covid19['new_tests'].replace(np.nan, '')
covid19['new_cases'] = covid19['new_cases'].replace(np.nan, '')
covid19['new_deaths'] = covid19['new_deaths'].replace(np.nan, '')
covid19['new_vaccinations'] = covid19['new_vaccinations'].replace(np.nan, '')
covid19['total_cases'] = covid19['total_cases'].replace(np.nan, '')
covid19['total_vaccinations'] = covid19['total_vaccinations'].replace(np.nan, '')
covid19['people_fully_vaccinated'] = covid19['people_fully_vaccinated'].replace(np.nan, '')
covid19['total_deaths'] = covid19['total_deaths'].replace(np.nan, '')


#######################################################################################
st.write('**Choose a country to Visualize COVID-19 numbers :**')

country = st.selectbox("Choose a country",covid19["location"].unique())

st.header(f"Select the data of your interesse  to visualize for {country}")
daily = st.selectbox("Select the option",('Total Cases', 'Total Deaths', 'Total Vaccinations'))
typ = st.radio("Select the type of Chart",("Line Chart","Scatter Chart"))


vaccine = alt.Chart(covid19[covid19["location"]==country]).encode(
x="date", y="new_vaccinations", tooltip=["date:T","location:N","new_vaccinations:Q"]).interactive()

t_cases = alt.Chart(covid19[covid19["location"]==country]).encode(
x="date",y="total_cases",tooltip=["date:T","location:N","total_cases:Q"]).interactive()

t_deaths = alt.Chart(covid19[covid19["location"]==country]).encode(
x="date",y="total_deaths",tooltip=["date:T","location:N","total_deaths:Q"]).interactive()

t_vaccinations = alt.Chart(covid19[covid19["location"]==country]).encode(
x="date",y="total_vaccinations",tooltip=["date:T","location:N","total_vaccinations:Q"]).interactive()

cases= alt.Chart(covid19[covid19["location"]==country],title="Scatter Chart",width=800,height=600).mark_circle(color='green').encode(
x ="date",
y ="total_cases",
size ="total_deaths",
color ="total_vaccinations",
tooltip=["date:T","location:N", "total_cases:Q","total_deaths:Q","total_vaccinations:Q"]
).interactive()



if daily =='Daily New Cases':
  if typ == 'Line Chart':
    st.altair_chart(new_cases.mark_line(color='firebrick'))
  else:
    st.altair_chart(new_cases.mark_circle(color='firebrick'))
elif daily =='Total Vaccinations':
  if typ == 'Line Chart':
    st.altair_chart(t_vaccinations.mark_line(color='green'))
  else:
    st.altair_chart(t_vaccinations.mark_circle(color='green'))
elif daily =='Daily New Deaths':
  if typ == 'Line Chart':
    st.altair_chart(new_deaths.mark_line(color='purple'))
  else:
    st.altair_chart(new_deaths.mark_circle(color='purple'))
elif daily =='Total Deaths':
  if typ == 'Line Chart':
    st.altair_chart(t_deaths.mark_line(color='yellow'))
  else:
    st.altair_chart(t_deaths.mark_circle(color='yellow'))
elif daily =='Total Cases':
  if typ == 'Line Chart':
    st.altair_chart(t_cases.mark_line(color='orange'))
  else:
    st.altair_chart(t_cases.mark_circle(color='orange'))
elif daily =='New Tests':
  if typ == 'Line Chart':
    st.altair_chart(new_tests.mark_line(color='red'))
  else:
    st.altair_chart(new_tests.mark_circle(color='red'))

if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [ ]:
from pyngrok import ngrok
url = ngrok.connect(port=8501)
url

ERROR:pyngrok.process:t=2025-05-11T04:31:06+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"



PyngrokNgrokError: The ngrok process errored on start: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
ngrok.kill()